TEST CHANGE

#### **Swan Project - Decision Trees**

In [3]:
# The big four libraries!
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Measure performance of our model.
from sklearn import metrics 

# Import the decision tree model 
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
## Import Logistic Regression from sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [4]:
swan = pd.read_csv('1 - Project Data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '1 - Project Data.csv'

In [ ]:
swan.head()

In [ ]:
swan.columns

In [ ]:
swan['Tenure Months']

In [ ]:
swan.shape

In [ ]:
swan.isnull().sum()

In [ ]:
swan['Churn Value'].value_counts()

In [ ]:
swan['Churn Reason'].value_counts()
# Could group these into three groups: Products, Swan Servies and Competitors. 

For the class imbalance: **model = DecisionTreeClassifier(class_weight=weights)**

In [ ]:
swan.Contract.unique()

In [ ]:
swan['Payment Method'].unique()

In [ ]:
swan['Monthly Charges'].mean()

In [ ]:
swan.dtypes

**Feature Engineering**

In [ ]:
def yes_no(column):
    x=column
    if x=='Yes':
        return 1
    else:
        return 0
        
products = ['Product dissatisfaction', 'Network reliability', 'Lack of self-service on Website', 'Limited range of services', 'Long distance charges', 'Lack of affordable download/upload speed' ]
swan_services = ['Attitude of support person', 'Attitude of service provider',   'Price too high', 'Service dissatisfaction',  'Extra data charges', 'Poor expertise of phone support', 'Poor expertise of online support']
competitor_services = ['Competitor offered higher download speeds', 'Competitor offered more data', 'Competitor made better offer', 'Competitor had better devices']

def churn_reason(reason):
    if reason in products:
        return 'Issue with products' 
    elif reason in swan_services:
        return 'Issue with Swan Services' 
    elif reason in competitor_services:
        return 'Competitor offered better services'
    else:
        return 'Other'

#### **What factors influence Churning?**

In [ ]:
def yes_no(column):
    x=column
    if x=='Yes':
        return 1
    else:
        return 0
    
def feature_eng(x):
    df=x.copy()
    yes_no_columns=['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines',
                    'Online Security', 'Online Backup', 'Device Protection', 
                    'Tech Support', 'Streaming TV', 'Streaming Movies', 'Paperless Billing']
    for column in yes_no_columns:
        df[column]=df[column].apply(yes_no)     
    #df['Churn Category'] = df['Churn Reason'].apply(churn_reason)
    encoding_columns=['Contract','Payment Method', 'Internet Service'] #'Churn Category']
    encoding_prefix=['Contract','Payment', 'Internet'] #'Churn'
    df= pd.get_dummies(data=df, 
                       columns=encoding_columns, 
                       prefix=encoding_prefix, 
                       dtype=int)
    df.drop(columns = ['Count', 'Country', 'State', 'CustomerID','City', 'Churn Reason', 'Churn Label', 'Lat Long'], inplace= True)
    df = df.drop(df[df['Total Charges'] == ' '].index)
    df.Gender = df.Gender.map({'Male':0, 'Female':1})
    return df

In [ ]:
# Correlation Table
feature_eng(swan).corr(numeric_only=True)

In [ ]:
(feature_eng(swan).corr(numeric_only=True)['Churn Value'].abs().sort_values(ascending=False))

In [ ]:
feature_eng(swan).columns

In [ ]:
plt.figure(figsize = (30, 20))
sns.heatmap(feature_eng(swan).corr(numeric_only = True),
            annot = True,
            fmt = '.2',
            cmap = 'rocket_r',
            mask = np.triu(feature_eng(swan).corr(numeric_only = True)),
            vmin = -1,
            vmax = 1)

plt.show()

#### **Logistic Regression Model**

In [ ]:
feature_eng(swan)

In [ ]:
df_model=feature_eng(swan).copy()

In [ ]:
df_model.head()

In [ ]:
df_model.loc[:,'Tenure Months':'Contract_Month-to-month']

In [ ]:
df_model.columns

In [ ]:
y = df_model['Churn Value'].copy()

In [ ]:
lr = LogisticRegression(max_iter=300, random_state=10)

In [ ]:
lr.fit(df_model, y)

In [ ]:
len(y) == df_model.shape[0]

In [ ]:
Features = ['Zip Code', 'Lat Long', 'Latitude', 'Longitude', 'Gender',
       'Senior Citizen', 'Partner', 'Dependents', 'Tenure Months',
       'Phone Service', 'Multiple Lines', 'Online Security', 'Online Backup',
       'Device Protection', 'Tech Support', 'Streaming TV', 'Streaming Movies',
       'Paperless Billing', 'Monthly Charges', 'Total Charges', 'Churn Label',
       'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'Payment_Bank transfer (automatic)',
       'Payment_Credit card (automatic)', 'Payment_Electronic check',
       'Payment_Mailed check', 'Internet_DSL', 'Internet_Fiber optic',
       'Internet_No']

In [ ]:
df_model.dtypes

In [ ]:
lr.fit(df_model, y)

In [ ]:
lr.predict_proba(df_model)

In [ ]:
df_model[['prob_churn', 'prob_not_churn']] = lr.predict_proba(df_model)

In [ ]:
df_model.head()

In [ ]:
df_model['y_pred'] = np.where(df_model['prob_churn']>.5, 1, 0)

In [ ]:
print(metrics.confusion_matrix(y, df_model['y_pred']))

In [ ]:
def get_results(actual, predicted):
    print("The confusion matrix for your predictions is:")
    print(metrics.confusion_matrix(actual, predicted), "\n")
    print(f'The accuracy of your model is: {metrics.accuracy_score(actual, predicted)}')
    print(f'The recall of your model is: {metrics.recall_score(actual, predicted)}')
    print(f'The precision of your model is: {metrics.precision_score(actual, predicted)}')
    print(f'The F1-score of your model is: {metrics.f1_score(actual, predicted)}')

In [ ]:
get_results(y, df_model['y_pred'])

In [ ]:
print(metrics.classification_report(y, df_model['y_pred']))

In [ ]:
metrics.ConfusionMatrixDisplay(metrics.confusion_matrix(y, df_model['y_pred'])).plot()
plt.show()